#### Understanding the Problem 
        Customer reviews and application  are longer and descriptive. if we check manually then it will take much longer time, so here comes the excellence of the Natural language processing, i'm building the text summarizer. 
    

#### Importing the Necessary Libraries

In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pylab as plt 
import seaborn as sns 

from bs4 import BeautifulSoup

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, LSTM, Input, Embedding, TimeDistributed, Concatenate

from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Reading the Dataset

In [9]:
dataset = pd.read_csv('dataset\Reviews.csv')
dataset = dataset[['Summary', 'Text']]
dataset.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
dataset.isna().sum()
dataset.dropna(axis=0, inplace=True)
dataset.drop_duplicates(keep='first')

Summary    27
Text        0
dtype: int64

In [15]:
len_dataset = len(dataset)
print(f"length of the dataset : {len_dataset}")

length of the dataset : 568427


#### Dataset Preprocessing

In [ ]:
contraction_expanding = 

In [ ]:
def text_preprocessing(text):
    text = text.lower()
    text = BeautifulSoup(text, 'lxml').text # removing html tags
    text = re.sub("'s\b", " ", text)